# 26 - File Formats

---

Data is usually stored in files, which are constructed according to a specific file format. Standardized file formats often are supported by Python using a particular module. In this chapter, I discuss some of more common file formats and the modules that support them.

---

## Comma-Separated Values (CSV)

Comma-Separated Values (CSV) is the most common text file format that is used for importing and exporting data to and from spreadsheets and databases. The general format says that each line contains one record (a record is a complete entity), listing each of the fields of the record in a specific order, separating the fields by commas. The first line of the file may or may not consist of names for the fields in the CSV file.

The code below loads and displays the contents of a typical CSV file that I enclosed with the course.

In [ ]:
fp = open( "pc_inventory.csv" )
print( fp.read().strip() )
fp.close()

Unfortunately, the CSV format is not standardized, and different software packages tend to use slightly different implementations of CSV files. However, over the years the different conventions used by the major software packages have converged to something of a standard, that is implemented in the Python `csv` module. The module supports "dialects" of CSV formats to handle files from different sources.

Instead of using the `csv` module, if you have to deal with an excentric CSV format that the module does not support, you can try to design your own interpretation of lines of the file using regular expressions. You can also try to design your own dialect. Neither option is very appealing.

### CSV `reader()`

The `csv` module contains a `reader()` function that provides access to a CSV file. The `reader()` function gets a file handle as argument, and returns an iterator that allows you to get the lines from the file, as a list with each of the fields as an element of the list. You should leave the file open while accessing it with `reader()`.

In [ ]:
from csv import reader

fp = open( "pc_inventory.csv", newline='' )
csvreader = reader( fp )
for line in csvreader:
    print( line )
fp.close()

The Python documentation recommends that if you use `reader()` on a file (and that is what you usually do), you specify a `newline=''` argument as extra argument when opening the file (I did this in the code above). This is necessary in case some of the text fields in the CSV file contain newline characters.

`reader()` takes extra arguments too. The most common ones are `delimiter=<character>`, which indicates which `<character>` is placed between different fields (default is "`,`"), and `quotechar=<character>`, which indicates which `<character>` is used to enclose strings with (default is "`"`"). 

### CSV `writer()`

Writing a CSV file is just a little bit harder than reading one. You create a file handle to a file that you open for writing ("`w`" mode), and use it as an argument when you call the `writer()` function from the `csv` module. The object that is returned from the `writer()` call has a method `writerow()` that you can call with a list of fields, that it then writes to the output file in CSV format.

The call to `writer()` can get the same arguments as the call to `reader()` can get, including specifying a `delimiter` and a `quotechar`. Moreover, you can a supply a `quoting=<quotemethod>` argument, that supports the following methods of quoting:

- `csv.QUOTE_ALL`, which encloses every field in quotation characters
- `csv.QUOTE_MINIMAL`, which only encloses fields in quotation characters if it is absolutely necessary (this is the default)
- `csv.QUOTE_NONNUMERIC`, which encloses fields in quotation characters if they are not integers or floats
- `csv.QUOTE_NONE`, which encloses no fields in quotation characters

Enclosing a string within quotation characters is generally only needed if the string contains exceptional characters, such as newlines or the same character that is used as delimiter. 

In [ ]:
from csv import writer

fp = open( "pc_writetest.csv", "w", newline='' )
csvwriter = writer( fp )
csvwriter.writerow( ["MOVIE", "RATING"] )
csvwriter.writerow( ["Monty Python and the Holy Grail", 8] )
csvwriter.writerow( ["Monty Python's Life of Brian", 8.5] )
csvwriter.writerow( ["Monty Python's Meaning of Life", 7] )
fp.close()

**Exercise**: After using the code above to create the file "pc_writetest.csv", open it and use `reader()` to list its contents.

In [ ]:
# Read what was written.


---

## Pickling

Suppose that you want to store a certain data structure in a file, for instance, a list of tuples. One way of doing that is to turn the tuples into strings and write those into the file, one line for every tuple. When you then later want to rebuild the data structure in a program, you read the file, unravel the lines, and reconstruct the list of tuples. As you can imagine, this encompasses a considerable amount of quite difficult code.

Fortunately, you do not have to write such code. Python offers a solution for storing data structures in files, including both structure and content, which is called "pickling". You can write the whole data structure to the file in one go, if you just open a *binary* file for writing, and call the function `dump()` from the `pickle` module with the data structure as first argument, and the file handle as second argument.

In [ ]:
from pickle import dump

cheeseshop = [ ("Roquefort", 12, 15.23), ("White Stilton", 25, 19.02), ("Cheddar", 5, 0.67) ]

fp = open( "pc_cheese.pck", "wb" )
dump( cheeseshop, fp )
fp.close()

print( "Cheeseshop was pickled" )

To read the contents of a pickle file, you use the function `load()` from the `pickle` module. `load()` gets a handle to the file as argument. Do not forget to open the file in binary mode. 

In [ ]:
from pickle import load

fp = open( "pc_cheese.pck", "rb" )
buffer = load( fp )
fp.close()

print( type( buffer ) )
print( buffer )

As you can see, `load()` restores the data structure completely.

Pickling works even for your own classes:

In [ ]:
from pickle import dump, load

class Point:
    def __init__( self, x, y ):
        self.x = x
        self.y = y
    def __repr__( self ):
        return "({},{})".format( self.x, self.y )
    
p = Point( 2, 5 )
fp = open( "pc_point.pck", "wb" )
dump( p, fp )
fp.close()

fp = open( "pc_point.pck", "rb" )
q = load( fp )
fp.close()

print( type( q ) )
print( q )

---

## JavaScript Object Notation (JSON)

JavaScript Object Notation (JSON) is a file format that is often used in modern applications, in particular those that communicate via web services. It is supported by many languages (JavaScript amongst them, of course). It is similar to pickling in the sense that it stores in-memory objects to files, retaining their structure. A difference with pickling is that JSON files are in human-readable format. 

The `json` module works equivalent to the `pickle` module, with a `dump()` function that writes data structures to a file, and a `load()` function to load data structures from a file. The file must be a text file, and not a binary file.

In [ ]:
from json import dump, load

cheeseshop = [ ("Roquefort", 12, 15.23), ("White Stilton", 25, 19.02), ("Cheddar", 5, 0.67) ]

fp = open( "pc_cheese.json", "w" )
dump( cheeseshop, fp )
fp.close()

fp = open( "pc_cheese.json", "r" )
buffer = load( fp )
fp.close()

print( type( buffer ) )
print( buffer )

Alternatives for `dump()` and `load()` are the functions `dumps()` and `loads()`, which do not get a file argument. Instead, `dumps()` gets no file argument at all, and just produces a string that contains the data structure in JSON format, while `loads()` gets a string as argument instead of a file, and loads the data structure from that string.

These functions can get many optional arguments that determine how exactly the data will be stored; for instance, you can set the `indent=` argument for `dump()` and `dumps()` to determine which indentation value will be used, and you can use arguments to sort the data in the dump. If you want to know more about this, consult the references.

A weakness of the `json` module is that it only supports the standard Python data structures. If you want to use it to store instances of classes of your own making, you have to find a way to convert your own classes to standard Python structures. The `json` module offers special `JSONencoder` and `JSONdecoder` classes to help you with that. It goes too far to discuss these here.

---

## HTML and XML

HTML and XML are standard formats that are used to display information on webpages. They consist of readable text files, with many instructions on formatting. It is a common task for data miners to "scrape" data from webpages. You can use regular expressions for that, but if the webpages are reasonably well-formatted, the "Beautiful Soup" module may help you out.

The Beautiful Soup module is named `bs4` in Python (naturally, `bs3` came before it, and it may get more updates later). It contains the `BeautifulSoup` class that you can use to load and interpret HTML and XML files. `bs4` is not part of the standard Python package; you have to install it separately, which is quite a hassle, unless you use a tool called `pip` which comes standard with Python 3.

There are alternative modules that can ease the pain of web scraping for you, notably `lxml`, but Beautiful Soup seems to be the most popular. 

Since all such modules require separate installations, I will not discuss them here. I only wish to indicate that if you need to do web scraping (and it is likely you have to do that at some point), you should check out some of the standard tools available for that before you delve into eccentric regular expression-design.

---

## What you learned

In this chapter, you learned about:

- Reading and writing CSV files using the `csv` module
- Pickling using the `pickle` module
- Reading and writing JSON files using the `json` module
- The availability of tools for web scraping

---

## Exercises

### Exercise 26.1

Open the file "pc_inventory.csv" and read its contents using `reader()`. Write the contents to a different CSV file, using a space as delimiter and a single quote as quotation character. Open the file you created as text file and display its contents to check them.

In [ ]:
# Reading and writing CSV.


### Exercise 26.2

Load the contents from the file "pc_inventory.csv", and put them in a list of lists (each line in the file being one list in the list of lists). Store the list in JSON format. Open the file you created as a text file and display its contents to examine them.

In [ ]:
# Reading CSV and writing JSON.
